## Генерируем данные

In [306]:
import numpy as np 
import pandas as pd
import os

In [307]:
def true_fun(x, a=np.pi, b = 0, f=np.sin): 
    x = np.atleast_1d(x)[:] 
    a = np.atleast_1d(a)
    
    f = lambda x:x 
    x = np.sum([ai*np.power(x, i+1) for i,ai in enumerate(a)],axis=0)

    return f(x+ b)

def noises(shape , noise_power):
    return np.random.randn(*shape) *noise_power 
                                            

def dataset(a, b, N = 250, x_max =1, noise_power = 0, seed = 42):
    np.random.seed(seed) 
    x = np.linspace(0,x_max,N)
    y_true = np.array([]) 
    f = None
    for f_ in np.append([], f): 
        y_true=np.append(y_true, true_fun(x, a, b, f_)) 
    y_true = y_true.reshape(-1,N).T
    y = y_true + noises(y_true.shape , noise_power) 
    y = y *3 + 10
    x = x * 10
    return {'Time': list([t[0] for t in np.atleast_2d(x).T]), 'Temperature': list([t[0] for t in y])}  

In [310]:
if(not os.path.exists("train")):
   os.mkdir("train")

if(not os.path.exists("test")):
   os.mkdir("test")

Пускай будет три тренировочных набора, один тестовый

In [311]:
train1 = dataset(a = [3,-1,-2], b = 1, N = 360, x_max =1.2, noise_power = 0.05)
pd.DataFrame(data=train1, columns=['Time', 'Temperature']).to_csv("train/train 1.csv", sep=',')

In [312]:
train2 = dataset(a = [3,-1,-2], b = 1, N = 360, x_max =1.2, noise_power = 0.15,seed = 45)
pd.DataFrame(train2, columns=['Time', 'Temperature']).to_csv("train/train 2.csv", sep=',')

In [313]:
train3 = dataset(a = [3,-1,-2], b = 1, N = 360, x_max =1.2,seed = 43)
pd.DataFrame(train3, columns=['Time', 'Temperature']).to_csv("train/train 3.csv", sep=',')

In [314]:
test = dataset(a = [3,-1,-2], b = 1, N = 360, x_max =1.2, noise_power = 0.05, seed = 41)
pd.DataFrame(test, columns=['Time', 'Temperature']).to_csv("test/test 1.csv", sep=',')

## Препроцессинг

In [315]:
from sklearn import preprocessing
import itertools
import numpy as np 
import pandas as pd
import os

In [316]:
path1 = 'test'
path2 = 'train'
is_first = True

for path, dirs, files in itertools.chain(os.walk(path1), os.walk(path2)):
    for filename in files:            
        data = pd.read_csv('{0}/{1}'.format(path,filename),index_col=0)
        array = np.array(data.Time).reshape(-1, 1)
        if (is_first):
            scaler = preprocessing.StandardScaler().fit(array)
            is_first = False
        data["Time"]  = scaler.transform(array)
        data.to_csv('{0}/{1}'.format(path,filename), sep=',')


## Обучение модели

In [317]:
from sklearn.linear_model import LinearRegression
import pickle
import numpy as np 
import pandas as pd
import os

In [318]:
def to_polynom(x, order = 1, add_bias = False):
    order_range = range( 0 if add_bias else 1, order+1,1)
    x = np.atleast_1d(x)[:]    
    out = np.array([])
    for i in order_range:
        out = np.append(out, np.power(x,i))
    return out.reshape(-1, x.size).T

In [319]:
path = 'train'
train_data = []
for path, dirs, files in os.walk(path):
    for filename in files:      
        train_data.append(pd.read_csv('{0}/{1}'.format(path,filename),index_col=0))

train_data = pd.concat(train_data, axis = 0,ignore_index=True)
X = np.array(train_data.Time).reshape(-1, 1)
y = np.array(train_data.Temperature).reshape(-1, 1)
x_pol = to_polynom(X, order = 3)
reg = LinearRegression().fit(x_pol, y)


In [321]:
filename = 'model.sav'
pickle.dump(reg, open(filename, 'wb'))

## Тестирование модели

In [326]:
from sklearn.linear_model import LinearRegression
import pickle

In [327]:
filename = 'model.sav'
loaded_model = pickle.load(open(filename, 'rb'))

In [328]:
result = loaded_model.score(x_pol, y)
print(result)

0.9762913072791282
